In [ ]:
import torch
import random
import copy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_wine

In [ ]:
wine = load_wine()
features =  13 # all features from dataset

X_train, X_test, y_train, y_test = train_test_split(
    wine.data[:, :features], 
    wine.target, 
    test_size=0.3,
    shuffle=True)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
class WineNet(torch.nn.Module):
    def __init__(self, n_input, n_hidden_neurons):
        super(WineNet, self).__init__()
        self.fc1 = torch.nn.Linear(n_input, n_hidden_neurons)
        self.activ1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.activ2 = torch.nn.Tanh()
        self.fc3 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.sm = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.fc2(x)
        x = self.activ2(x)
        x = self.fc3(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x

In [ ]:
n_input =  13 # all features
n_hidden =  50
wine_net = WineNet(n_input, n_hidden)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(wine_net.parameters(), lr=1.0e-3)

In [ ]:
best_wine_net = None
best_net_result = 0

In [ ]:
batch_size =  10 # choose different batch sizes

for epoch in range(2000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index + batch_size]
        
        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = wine_net.forward(x_batch)
        
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
        
    if epoch % 20 == 0:
        test_preds = wine_net.forward(X_test)
        test_preds = test_preds.argmax(dim=1)
        accuracy = (test_preds == y_test).float().mean()
        if accuracy >= best_net_result:
            print(accuracy)
            best_net_result = accuracy
            best_wine_net = copy.deepcopy(wine_net)


tensor(0.3704)
tensor(0.6296)
tensor(0.7778)
tensor(0.8333)
tensor(0.8519)
tensor(0.8704)
tensor(0.8889)
tensor(0.9074)
tensor(0.9074)
tensor(0.9074)
tensor(0.9074)
tensor(0.9074)
tensor(0.9074)
tensor(0.9074)
tensor(0.9074)
tensor(0.9074)
tensor(0.9259)


In [ ]:
test_preds = best_wine_net.forward(X_test)
test_preds = test_preds.argmax(dim=1)
print((test_preds == y_test).float().mean())

tensor(0.9259)


In [ ]:
print(best_wine_net.fc1.in_features, np.asarray((test_preds == y_test).float().mean()) > 0.8)
# need to get 13 True

13 True


In [3]:
import torch

# Создаем входной массив из двух изображений RGB 3*3
input_images = torch.tensor(
      [[[[0,  1,  2],
         [3,  4,  5],
         [6,  7,  8]],

        [[9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]],


       [[[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44]],

        [[45, 46, 47],
         [48, 49, 50],
         [51, 52, 53]]]])


def get_padding2d(input_images):
    padded_images = [] # добавить нулей с четырех сторон каждого изображения
    for image_num in range(input_images.shape[3]):
        padded_images.append([])
        for depth in range(input_images.shape[2]):
            padded_images[image_num].append([])
            padded_images[image_num][depth].append([0] * (input_images.shape[1] + 2))
            for row in input_images[:, :, depth, image_num]:
                padded_images[image_num][depth].append([0] + list(row) + [0])
            padded_images[image_num].append([0] * (input_images.shape[1] + 2))
    return torch.Tensor(padded_images)


correct_padded_images = torch.tensor(
       [[[[0.,  0.,  0.,  0.,  0.],
          [0.,  0.,  1.,  2.,  0.],
          [0.,  3.,  4.,  5.,  0.],
          [0.,  6.,  7.,  8.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0.,  9., 10., 11.,  0.],
          [0., 12., 13., 14.,  0.],
          [0., 15., 16., 17.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0., 18., 19., 20.,  0.],
          [0., 21., 22., 23.,  0.],
          [0., 24., 25., 26.,  0.],
          [0.,  0.,  0.,  0.,  0.]]],


        [[[0.,  0.,  0.,  0.,  0.],
          [0., 27., 28., 29.,  0.],
          [0., 30., 31., 32.,  0.],
          [0., 33., 34., 35.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0., 36., 37., 38.,  0.],
          [0., 39., 40., 41.,  0.],
          [0., 42., 43., 44.,  0.],
          [0.,  0.,  0.,  0.,  0.]],

         [[0.,  0.,  0.,  0.,  0.],
          [0., 45., 46., 47.,  0.],
          [0., 48., 49., 50.,  0.],
          [0., 51., 52., 53.,  0.],
          [0.,  0.,  0.,  0.,  0.]]]])

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# print(torch.allclose(get_padding2d(input_images), correct_padded_images))

In [4]:
padd = get_padding2d(input_images)
padd

ValueError: ignored